# Loading stock prices with vnstock

In [1]:
from vnstock import Quote
import pandas as pd

# Define the symbols you want to fetch data for
symbols = ['REE', 'FMC', 'DHC']
print(f"Fetching historical price data for: {symbols}")

# Dictionary to store historical data for each symbol
all_historical_data = {}

# Set date range
start_date = '2024-01-01'
end_date = '2025-03-19'
interval = '1D'

# Fetch historical data for each symbol
for symbol in symbols:
    try:
        print(f"\nProcessing {symbol}...")
        quote = Quote(symbol=symbol)
        
        # Fetch historical price data
        historical_data = quote.history(
            start=start_date,
            end=end_date,
            interval=interval,
            to_df=True
        )
        
        if not historical_data.empty:
            all_historical_data[symbol] = historical_data
            print(f"Successfully fetched {len(historical_data)} records for {symbol}")
        else:
            print(f"No historical data available for {symbol}")
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Export all historical data to a single CSV file
if all_historical_data:
    # Create a combined DataFrame with all data
    combined_data = pd.DataFrame()
    
    for symbol, data in all_historical_data.items():
        if not data.empty:
            # Make a copy of the data and rename columns to include symbol
            temp_df = data.copy()
            # Keep 'time' column as is for merging
            for col in temp_df.columns:
                if col != 'time':
                    temp_df.rename(columns={col: f'{symbol}_{col}'}, inplace=True)
            
            if combined_data.empty:
                combined_data = temp_df
            else:
                combined_data = pd.merge(combined_data, temp_df, on='time', how='outer')
    
    # Sort by time
    if not combined_data.empty:
        combined_data = combined_data.sort_values('time')
        
        # Display sample of combined data
        print("\nSample of combined data:")
        print(combined_data.head(3))
        
        # Export combined data to CSV
        combined_csv_filename = 'all_historical_data.csv'
        combined_data.to_csv(combined_csv_filename, index=False, encoding='utf-8-sig')
        print(f"\nAll historical data exported to {combined_csv_filename}")
    
    # Also create a combined DataFrame for close prices only (for comparison purposes)
    combined_prices = pd.DataFrame()
    
    for symbol, data in all_historical_data.items():
        if not data.empty:
            # Extract time and close price
            temp_df = data[['time', 'close']].copy()
            temp_df.rename(columns={'close': f'{symbol}_close'}, inplace=True)
            
            if combined_prices.empty:
                combined_prices = temp_df
            else:
                combined_prices = pd.merge(combined_prices, temp_df, on='time', how='outer')
    
    # Sort by time
    if not combined_prices.empty:
        combined_prices = combined_prices.sort_values('time')
        
        # Export combined close prices to CSV
        combined_close_csv_filename = 'combined_close_prices.csv'
        combined_prices.to_csv(combined_close_csv_filename, index=False, encoding='utf-8-sig')
        print(f"Combined close price data exported to {combined_close_csv_filename}")
else:
    print("No historical data was fetched for any symbol.")

Phiên bản Vnstock 3.2.4 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnstock --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 3.2.2

Fetching historical price data for: ['REE', 'FMC', 'DHC']

Processing REE...
Successfully fetched 300 records for REE

Processing FMC...
Successfully fetched 300 records for FMC

Processing DHC...
Successfully fetched 300 records for DHC

Sample of combined data:
        time  REE_open  REE_high  REE_low  REE_close  REE_volume  FMC_open  \
0 2024-01-02     48.54     49.21    48.20      48.54      779590     42.97   
1 2024-01-03     48.54     48.79    48.20      48.62      376152     43.54   
2 2024-01-04     48.88     49.64    48.54      48.71      899128     43.88   

   FMC_high  FMC_low  FMC_close  FMC_volume  DHC_open  DHC_high  DHC_low  \
0     44.17    42.97      43.78       18200     36.32     36.46    35.94   
1     43.88    43.50      43.69       26222     36.08     36.41    36.08   
2     44.07    43.64      43.83       19700     36.36     37.50    36.32   

   DHC_close  DHC_volume  
0      36.08      118392  
1      36.36      163177  
2      37.08      319156  

All histo

In [2]:
#print(combined_prices.head())
#print(combined_data.head())

In [3]:
# Set the time column as index and ensure it's datetime format
combined_prices_indexed = combined_prices.copy()
combined_prices_indexed['time'] = pd.to_datetime(combined_prices_indexed['time'])
combined_prices_indexed.set_index('time', inplace=True)

# Calculate daily returns for each stock
returns_df = pd.DataFrame(index=combined_prices_indexed.index)
for symbol in symbols:
    column_name = f'{symbol}_close'
    returns_df[symbol] = combined_prices_indexed[column_name].pct_change()

# Drop the first row which will have NaN values due to pct_change()
returns_df = returns_df.dropna()
# Create an equal-weighted portfolio
portfolio_returns = returns_df.mean(axis=1)


In [4]:
# 1. Make a copy of the combined_prices DataFrame
prices_df = combined_prices.copy()

# 2. Convert the 'time' column to datetime if it's not already
prices_df['time'] = pd.to_datetime(prices_df['time'])

# 3. Set the 'time' column as the index
prices_df.set_index('time', inplace=True)

# 4. Extract only the close price columns and rename them to just the symbol names
close_price_columns = [col for col in prices_df.columns if '_close' in col]
prices_df = prices_df[close_price_columns]
prices_df.columns = [col.replace('_close', '') for col in close_price_columns]

# 5. Make sure there are no NaN values
prices_df = prices_df.dropna()
print(prices_df.head())

              REE    FMC    DHC
time                           
2024-01-02  48.54  43.78  36.08
2024-01-03  48.62  43.69  36.36
2024-01-04  48.71  43.83  37.08
2024-01-05  48.45  43.88  37.98
2024-01-08  47.95  43.88  39.50


In [5]:
risk_free_rate=0.02
risk_aversion=1

In [6]:
from pypfopt.expected_returns import returns_from_prices
log_returns=False
returns = returns_from_prices(prices_df, log_returns=log_returns)
returns.head()

,REE,FMC,DHC
time,,,
2024-01-03,0.001648,-0.002056,0.007761
2024-01-04,0.001851,0.003204,0.019802
2024-01-05,-0.005338,0.001141,0.024272
2024-01-08,-0.010320,0.000000,0.040021
2024-01-09,-0.012304,0.013218,-0.004810


# Market Neutral Portfolio

In [12]:
from vnstock import Vnstock

# Khởi tạo đối tượng
vnstock = Vnstock()

# Lấy dữ liệu lịch sử hợp đồng tương lai VN30F1M (ví dụ)
df = vnstock.stock(symbol=symbol, source=source).quote.history(start=start_date, end=end_date, interval=interval)

print(df.head())


2025-05-07 11:18:48 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


        time    open    high     low   close  volume
0 2023-01-03   999.9  1045.5   999.8  1045.5  318703
1 2023-01-04  1046.5  1054.0  1039.0  1042.7  341378
2 2023-01-05  1044.2  1061.0  1043.6  1053.8  262657
3 2023-01-06  1053.2  1067.0  1046.6  1051.0  333870
4 2023-01-09  1059.0  1063.6  1050.2  1053.6  266210


In [21]:
from vnstock import Vnstock
symbol='VN30F1M'
source='VCI'
#stock = Vnstock().stock(symbol=symbol, source=source)

derivatives_prices=vnstock.stock(symbol=symbol, source=source).quote.history(start=start_date, end=end_date, interval=interval)
derivatives_prices.set_index('time', inplace=True)
derivatives_returns= returns_from_prices(derivatives_prices['close'], log_returns=log_returns)
#derivatives_returns.dropna(inplace=True)
derivatives_returns = derivatives_returns.ffill()
derivatives_prices.head()

2025-05-07 11:24:13 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


,open,high,low,close,volume
time,,,,,
2024-01-02,1138.5,1141.8,1131.0,1133.5,167307
2024-01-03,1127.2,1149.8,1126.7,1148.3,173569
2024-01-04,1146.0,1171.0,1145.5,1156.5,260804
2024-01-05,1156.1,1166.1,1154.2,1166.0,199452
2024-01-08,1166.1,1173.3,1160.6,1162.0,182329


In [14]:
derivatives_returns = pd.DataFrame(derivatives_returns)
derivatives_returns.columns = ['VN30F1M']
derivatives_returns

,VN30F1M
time,
2024-01-03,0.013057
2024-01-04,0.007141
2024-01-05,0.008214
2024-01-08,-0.003431
2024-01-09,0.000861
...,...
2025-03-13,-0.002161
2025-03-14,-0.000361
2025-03-17,0.003827


In [15]:
# Combine using inner join (only keeps dates that exist in both)
market_neutral_returns = pd.concat([returns, derivatives_returns], axis=1)
market_neutral_returns = market_neutral_returns.dropna()
market_neutral_returns

,REE,FMC,DHC,VN30F1M
time,,,,
2024-01-03,0.001648,-0.002056,0.007761,0.013057
2024-01-04,0.001851,0.003204,0.019802,0.007141
2024-01-05,-0.005338,0.001141,0.024272,0.008214
2024-01-08,-0.010320,0.000000,0.040021,-0.003431
2024-01-09,-0.012304,0.013218,-0.004810,0.000861
...,...,...,...,...
2025-03-13,-0.021680,-0.020387,-0.011958,-0.002161
2025-03-14,-0.018006,0.003122,-0.007564,-0.000361
2025-03-17,-0.001410,-0.004149,0.003049,0.003827


In [16]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov
from pypfopt.efficient_frontier import EfficientFrontier
mu_tral=mean_historical_return(market_neutral_returns, log_returns=log_returns )
S2=sample_cov(market_neutral_returns)
market_neutral_weights = (-1, 1)
market_neutral_portfolio = EfficientFrontier(mu_tral, S2, weight_bounds=(-1, 1)) #allowing for shorting index derivatives

Some returns are NaN. Please check your price data.
Some returns are infinite. Please check your price data.
invalid value encountered in reduce


In [ ]:
market_neutral_portfolio.max_sharpe(risk_free_rate=risk_free_rate)

In [23]:
market_neutral_returns.to_excel("market_neutral_returns.xlsx")